In [1]:
from loaders.cognicity_loader import CognicityLoader
# from image_recognition.aws_recog import AwsLabeler
# from image_recognition.goog_recog import GoogleLabeler
#import chennai_config
#config = chennai_config.config

import jakarta_config
config = jakarta_config.config
from learners.svm_learner import SvmLearner

# from nlp.bow_labeler import BowLabeler
# learn = SvmLearner(config, CognicityLoader, BowLabeler)


# from image_recognition.goog_recog import GoogleLabeler
# learn = SvmLearner(config, CognicityLoader, GoogleLabeler)

from image_recognition.aws_recog import AwsLabeler
learn = SvmLearner(config, CognicityLoader, AwsLabeler)

import random
validation_set = set(random.sample(config["flood_pkeys"], 5))
validation_set = validation_set.union(set(random.sample(config["no_flood_pkeys"], 5)))
params = {"T":1000}
learn.train( params, validation_set)

2019-08-19 09:54:42,574 - DEBUG - CognicityLoader constructed
2019-08-19 09:54:42,576 - DEBUG - AwsLabeler constructed
2019-08-19 09:54:42,577 - DEBUG - CognicityLoader constructed
2019-08-19 09:54:42,577 - DEBUG - AwsLabeler constructed
2019-08-19 09:54:42,579 - DEBUG - logging from: default_jakarta_data/aws_labels_default.p
2019-08-19 09:54:58,008 - INFO - Num Correct 5 Out of 10
2019-08-19 09:54:58,009 - INFO - Val score: 0.5


(array([[ 0.        ,  0.        , 29.2573452 , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         34.68480301, 38.1450882 ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]),
 array([[-1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1.]]))

In [2]:
learn.val_data_w_pkey[0,:]

array([  562.,  3800., 12900., 14699., 19880., 23459., 23784., 24985.,
       34949., 36565.])

In [3]:
val = learn.val_data_w_pkey[1:,:].T
learn.clf.predict(learn.val_data_w_pkey[1:,:].T)

array([ 1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.])

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

def run_grid_search(learn):
    
    dat = learn.t_data_w_pkey[1:,:].T
    lab = learn.t_labels[0, :]
    # learn.clf.decision_function(dat)
    
    param_grid = [
      {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
      {'C': [1, 10, 100, 1000], 'kernel': ['poly'], 'degree':[2, 3, 4, 5]},
      {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
    ]
    
    cv = GridSearchCV(svm.SVC(), param_grid, cv=5, n_jobs=5)
    
    cv.fit(dat, lab)
    sorted(cv.cv_results_.keys())
    cv.cv_results_
    
    cv.best_estimator_
    cv.best_score_
    return cv

In [5]:
from loaders.cognicity_loader import CognicityLoader
from learners.svm_learner import SvmLearner

# import chennai_config
# config = chennai_config.config

import jakarta_config
config = jakarta_config.config

le = []

import random
validation_set = set(random.sample(config["flood_pkeys"], 5))
validation_set = validation_set.union(set(random.sample(config["no_flood_pkeys"], 5)))
params = {"T":1000}

from image_recognition.goog_recog import GoogleLabeler
l = SvmLearner(config, CognicityLoader, GoogleLabeler)
l.train(params, validation_set)
le.append(l)

from image_recognition.aws_recog import AwsLabeler
l = SvmLearner(config, CognicityLoader, AwsLabeler)
l.train(params, validation_set)
le.append(l)

from nlp.bow_labeler import BowLabeler
l = SvmLearner(config, CognicityLoader, BowLabeler)
l.train(params, validation_set)
le.append(l)

result = []
for each in le:
    grid_search = run_grid_search(each)
    print(grid_search.best_estimator_)
    result.append(grid_search)

2019-08-19 09:55:06,093 - DEBUG - CognicityLoader constructed
2019-08-19 09:55:06,094 - DEBUG - GoogleLabeler constructed
2019-08-19 09:55:06,095 - DEBUG - CognicityLoader constructed
2019-08-19 09:55:06,096 - DEBUG - GoogleLabeler constructed
2019-08-19 09:55:14,055 - INFO - Num Correct 8 Out of 9
2019-08-19 09:55:14,056 - INFO - Val score: 0.8888888888888888
2019-08-19 09:55:17,576 - DEBUG - CognicityLoader constructed
2019-08-19 09:55:17,577 - DEBUG - AwsLabeler constructed
2019-08-19 09:55:17,578 - DEBUG - CognicityLoader constructed
2019-08-19 09:55:17,578 - DEBUG - AwsLabeler constructed
2019-08-19 09:55:17,579 - DEBUG - logging from: default_jakarta_data/aws_labels_default.p
2019-08-19 09:55:33,356 - INFO - Num Correct 5 Out of 9
2019-08-19 09:55:33,357 - INFO - Val score: 0.5555555555555556
2019-08-19 09:55:40,981 - DEBUG - CognicityLoader constructed
2019-08-19 09:55:40,982 - DEBUG - CognicityLoader constructed
2019-08-19 09:56:16,580 - INFO - Num Correct 8 Out of 10
2019-08-1

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


/home/abrahamq/timeseries-analysis/_python/lib/python3.5/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=2, gamma='auto_deprecated',
    kernel='poly', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [8]:
for res in result:
    print(res.best_score_)
    print(res.best_estimator_)
    
# for chennai data: goog, aws, bow: cv =10
# 0.7572254335260116
# SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
#     decision_function_shape='ovr', degree=2, gamma='auto_deprecated',
#     kernel='poly', max_iter=-1, probability=False, random_state=None,
#     shrinking=True, tol=0.001, verbose=False)
# 0.7630057803468208
# SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
#     decision_function_shape='ovr', degree=4, gamma='auto_deprecated',
#     kernel='poly', max_iter=-1, probability=False, random_state=None,
#     shrinking=True, tol=0.001, verbose=False)
# 0.7745664739884393
# SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
#     decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
#     max_iter=-1, probability=False, random_state=None, shrinking=True,
#     tol=0.001, verbose=False)

# jakarta goog, aws, bow cv=10
# 0.6529968454258676
# SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
#     decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
#     max_iter=-1, probability=False, random_state=None, shrinking=True,
#     tol=0.001, verbose=False)
# 0.6421811626858945
# SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
#     decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
#     kernel='poly', max_iter=-1, probability=False, random_state=None,
#     shrinking=True, tol=0.001, verbose=False)
# 0.6768814781433078
# SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
#     decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
#     kernel='linear', max_iter=-1, probability=False, random_state=None,
#     shrinking=True, tol=0.001, verbose=False)
# 

0.6491310474401127
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
0.6341005166744951
SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=2, gamma='auto_deprecated',
    kernel='poly', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
0.6688311688311688
SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [7]:
cv.best_estimator_
# for jakarta aws data:
# SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
#     decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
#     kernel='linear', max_iter=-1, probability=False, random_state=None,
#     shrinking=True, tol=0.001, verbose=False)

# cv.best_score_ = 0.5971158179360072

NameError: name 'cv' is not defined

In [ ]:
cv.best_score_